# Fairness Analysis Master Notebook

This notebook orchestrates all phases of the fairness analysis pipeline.

**Usage:**
1. Set the `USERNAME` below
2. Run all cells to execute the complete analysis
3. Find the report at `data/other-users/{username}/report/report.html`

In [ ]:
# Configuration
USERNAME = "Jacob14GMA"  # Chess.com username to analyze # samayraina
DAYS_BACK = 0  # Days of history (0 = all)
TIME_CLASSES = ["rapid", "blitz"]  # Time controls to include
VALIDATE_OPPONENTS = True  # Check opponent ban status
MAX_GAMES_FOR_DEEP_ANALYSIS = 50  # Max games for engine analysis

# Advanced options
ENGINE_THREADS = 4
ENGINE_HASH_MB = 512
ANALYSIS_DEPTHS = [5, 10, 20]

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')
from pathlib import Path
from datetime import datetime

try:
    import papermill as pm
    PAPERMILL_AVAILABLE = True
except ImportError:
    PAPERMILL_AVAILABLE = False
    print("Warning: Papermill not installed. Install with: pip install papermill")
    print("Falling back to %run magic for notebook execution.")

# Paths
PHASES_DIR = Path("phases")
OUTPUT_BASE = Path("..") / "data" / "other-users" / USERNAME.lower() / "report"
EXECUTED_DIR = OUTPUT_BASE / "executed_notebooks"
EXECUTED_DIR.mkdir(parents=True, exist_ok=True)

print(f"Analyzing: {USERNAME}")
print(f"Output directory: {OUTPUT_BASE.resolve()}")

In [ ]:
def run_phase(phase_notebook: str, parameters: dict = None):
    """Run a phase notebook with parameters."""
    input_path = PHASES_DIR / phase_notebook
    output_path = EXECUTED_DIR / phase_notebook
    
    if not input_path.exists():
        print(f"ERROR: {input_path} not found")
        return False
    
    print(f"\n{'='*60}")
    print(f"Running: {phase_notebook}")
    print(f"{'='*60}")
    
    if PAPERMILL_AVAILABLE:
        try:
            pm.execute_notebook(
                str(input_path),
                str(output_path),
                parameters=parameters or {},
            )
            print(f"Completed: {phase_notebook}")
            return True
        except Exception as e:
            print(f"ERROR in {phase_notebook}: {e}")
            return False
    else:
        # Fallback: just note that we'd run it
        print(f"Would run {input_path} with parameters: {parameters}")
        print("Install papermill for actual execution: pip install papermill")
        return True

In [ ]:
# Phase 1: Data Collection
# Note: days_back=0 is treated as "all history" by the notebook
run_phase("01_data_collection.ipynb", {
    "username": USERNAME,
    "days_back": DAYS_BACK,
    "time_classes": TIME_CLASSES,
    "validate_opponents": VALIDATE_OPPONENTS,
})

In [ ]:
# Phase 2: Quick Analysis
run_phase("02_quick_analysis.ipynb", {
    "username": USERNAME,
})

In [ ]:
# Phase 3: Game Prioritization
run_phase("03_game_prioritization.ipynb", {
    "username": USERNAME,
    "min_suspicion_score": 3,
    "max_games_to_analyze": MAX_GAMES_FOR_DEEP_ANALYSIS,
})

In [ ]:
# Phase 4a: Engine Analysis (Multi-Depth Stockfish)
run_phase("04a_engine_analysis.ipynb", {
    "username": USERNAME,
    "analysis_depths": ANALYSIS_DEPTHS,
    "engine_threads": ENGINE_THREADS,
    "engine_hash_mb": ENGINE_HASH_MB,
})

In [ ]:
# Phase 4b: Ken Regan Analysis
run_phase("04b_regan_analysis.ipynb", {
    "username": USERNAME,
    "z_score_threshold": 2.0,
})

In [ ]:
# Phase 4c: Tablebase Analysis
run_phase("04c_tablebase_analysis.ipynb", {
    "username": USERNAME,
    "max_pieces": 7,
})

In [ ]:
# Phase 5: Time Analysis
run_phase("05_time_analysis.ipynb", {
    "username": USERNAME,
})

In [ ]:
# Phase 6: Maia2 Humanness Analysis
run_phase("06_maia2_analysis.ipynb", {
    "username": USERNAME,
})

In [ ]:
# Phase 7: Cheater Comparison
run_phase("07_cheater_comparison.ipynb", {
    "username": USERNAME,
})

In [ ]:
# Phase 8: Final Report
run_phase("08_final_report.ipynb", {
    "username": USERNAME,
    "report_title": f"Fairness Analysis: {USERNAME}",
})

In [ ]:
# Summary
print("\n" + "="*60)
print("ANALYSIS COMPLETE")
print("="*60)
print(f"\nPlayer: {USERNAME}")
print(f"Report: {(OUTPUT_BASE / 'report.html').resolve()}")
print(f"Executed notebooks: {EXECUTED_DIR.resolve()}")

# Try to display link
report_path = OUTPUT_BASE / "report.html"
if report_path.exists():
    from IPython.display import HTML, display
    display(HTML(f'<a href="{report_path}" target="_blank">Open Report</a>'))